In [ ]:
# GRU model
import pandas as pd
import numpy as np
from dataset import LABELS
from sklearn.preprocessing import LabelEncoder

In [1]:
train = pd.read_pickle('extracted_features\\features_training.pkl')
val = pd.read_pickle('extracted_features\\features_validation.pkl')

X_train = np.array([x[0] for x in train])
y_train = np.array([x[1] for x in train])
X_val = np.array([x[0] for x in val])
y_val = np.array([x[1] for x in val])

# Create a label encoder object
label_encoder = LabelEncoder()

# Fit the label encoder using your labels (combine both train and val labels if they have different classes)
label_encoder.fit(np.concatenate((y_train, y_val)))

# Transform your string labels to integer labels for both training and validation sets
y_train_encoded = label_encoder.transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dropout, Dense, Bidirectional, TimeDistributed, BatchNormalization, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D

results = pd.DataFrame(columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])

for gru_units in [64, 128, 256]:
    for dropout_rate in [0.2, 0.4, 0.6]:
        for batch_size in [32, 64, 128]:
            for epochs in [10, 20, 30]:
                num_classes = len(y_train_encoded)  # Number of unique classes in your dataset
                input_shape = (39, 44)

                model = Sequential([
                    Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=input_shape),
                    BatchNormalization(),
                    MaxPooling1D(pool_size=2),
                    Dropout(dropout_rate),
                    Bidirectional(GRU(gru_units, return_sequences=True)),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    Bidirectional(GRU(gru_units, return_sequences=True)),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    GRU(gru_units, return_sequences=True),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    GRU(gru_units, return_sequences=True),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    TimeDistributed(Dense(256, activation='relu')),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    TimeDistributed(Dense(128, activation='relu')),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    TimeDistributed(Dense(64, activation='relu')),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    GlobalMaxPooling1D(),
                    Dense(512, activation='relu'),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    Dense(256, activation='relu'),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    Dense(128, activation='relu'),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    Dense(num_classes, activation='softmax')
                ])

                # Compile the model
                model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                # Train the model with your training set and validate it with your validation set
                epochs = epochs
                batch_size = batch_size
                history = model.fit(X_train, y_train_encoded, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val_encoded))

                results = np.concatenate((results, pd.DataFrame([[gru_units, dropout_rate, epochs, batch_size, history.history['loss'][-1], history.history['loss'], history.history['accuracy'][-1], history.history['accuracy'], history.history['val_loss'][-1], history.history['val_loss'], history.history['val_accuracy'][-1], history.history['val_accuracy']]], 
                                                                    columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])), axis=0)



Epoch 1/10
214/214 [==============================] - 29s 69ms/step - loss: 6.9579 - accuracy: 0.0613 - val_loss: 3.6162 - val_accuracy: 0.0596
Epoch 2/10
214/214 [==============================] - 12s 56ms/step - loss: 2.8994 - accuracy: 0.1918 - val_loss: 2.5313 - val_accuracy: 0.2439
Epoch 3/10
214/214 [==============================] - 13s 60ms/step - loss: 2.4038 - accuracy: 0.2900 - val_loss: 2.0998 - val_accuracy: 0.3616
Epoch 4/10
214/214 [==============================] - 12s 57ms/step - loss: 2.0500 - accuracy: 0.3930 - val_loss: 1.8843 - val_accuracy: 0.4451
Epoch 5/10
214/214 [==============================] - 13s 62ms/step - loss: 1.8284 - accuracy: 0.4449 - val_loss: 1.6862 - val_accuracy: 0.4878
Epoch 6/10
214/214 [==============================] - 12s 56ms/step - loss: 1.7079 - accuracy: 0.4755 - val_loss: 1.6471 - val_accuracy: 0.5157
Epoch 7/10
214/214 [==============================] - 13s 61ms/step - loss: 1.6029 - accuracy: 0.5110 - val_loss: 1.7111 - val_accuracy:

In [4]:
from keras.optimizers import Adam

opt = Adam(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 37, 128)           17024     
                                                                 
 batch_normalization (BatchN  (None, 37, 128)          512       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 18, 128)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 18, 128)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 18, 256)          198144    
 l)                                                              
                                                        

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
results.to_pickle('results\\model_gru')